In [1]:
# Transferred that functions into separate file
# IN otder to keep the code clean

from misc import loadProperties, loadWEKA

In [2]:
props = loadProperties('submitActionClass.properties')
(data, attr) = loadWEKA('youTubeLocationIDWeka.csv', limit=0)
data[:2]

[['1892013124614', '', '', '', '', 'East Contoy Island Mexico', '2c'],
 ['1892013125327', '', '', '', '', 'East Contoy Island Mexico', '2c']]

### Matching YT video titles with their location
Naive way looping through the title and description word by word, trying to match the location:

In [3]:
import spacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

Attempt to adjust a pipeline for better location recognition:

In [4]:
nlp = spacy.load("en_core_web_sm")

locations = [x for x in props] # Array of known locations from .properties
location_patterns = list(nlp.pipe(locations))

matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
matcher.add("LOCATION", None, *location_patterns)

# Define the custom component
def location_component(doc):
    # Apply the matcher to the doc
    matches = matcher(doc)
    # Create a Span for each match and assign the label 'LOCATION'
    # Overwrite the doc.ents with the matched spans
    doc.ents = [Span(doc, start, end, label="LOCATION") for match_id, start, end in matches]
    return doc

# Add the component to the pipeline after the 'ner' component
nlp.add_pipe(location_component, before='ner')

# Just an example of recognition
doc = nlp("Snorkeling in Thudufushi Thailand in Mexico")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Thudufushi', 'GPE'), ('Thailand', 'LOCATION'), ('Mexico', 'GPE')]


In [5]:
# Uses spacy to look up for location in a strings. Returns Array of matches
def nlpLocation(string):
    spacy_mathc = []
    
    for ent in nlp(string).ents:
        if ent.label_ == "GPE" or ent.label_ == "LOCATION":
            spacy_mathc.append(ent.text.lower())

    return spacy_mathc

In [6]:
def matchLocation(data, props, description=False, output=False):
    entries_processed = 0
    matched = 0
    spacy_found = 0
    
    for item in data:
        # Working with YT videos only
        if item[1]:
            naive_match = [] # [[matches_in_title], [mathces_in_descr]]
            spacy_match = [] # [[identified_in_title], [identified_in_descr], [identified_in_tags]]
            
            # Going through title word by word
            title_splitted = item[2].split()
            title_match = []
            for word in title_splitted:
                try:
                    if (props[word.lower()]):
                        title_match.append(word)
                except:
                    pass
                
            naive_match.append(title_match)
            
            # Use spacy if location was not matched in a title
            spacy_match.append(nlpLocation(item[2].replace("'","")))
            
            # Parse description as well
            if description:
                # Going through description word by word
                descr_splitted = item[4].split()
                descr_match = []
                for word in descr_splitted:
                    try:
                        if (props[word.lower()]):
                            descr_match.append(word)
                    except:
                        pass
                    
                naive_match.append(descr_match)
                    
                # Use spacy if location was not matched in a description
                spacy_match.append(nlpLocation(item[4].replace("'","")))
                
            # Parse tags
            spacy_match.append(nlpLocation(item[3].replace("'","")))
    
            # Counting statistics
            entries_processed += 1
            matched += 1 if sum(len(x) for x in naive_match)>0 else 0
            spacy_found += 1 if sum(len(x) for x in spacy_match)>0 else 0
            
            if output:
                print(item)
                print("Naive match", naive_match)
                print("Spacy match", spacy_match)
                print()

    print("Processed ", entries_processed, " videos")
    print("Naively matched ", matched, " locations")
    print("Identified by spacy ", spacy_found, " locations")
    
matchLocation(data[:11800], props, description=True, output=True) # Pass data[:11800] to see a small bunch of results

['01e44824-117c-42c4-90aa-09871b745e16', 'https://www.youtube.com/watch?v=J6JKpXsBWuk', 'Snorkeling in Thudufushi  Whale Shark Manta Sharks Turtles', 'MaldiveThudufushiAri AtollAtollo di AriWhale sharkSqualo balenaSharksSqualiMurenaMorayStingrayTrigoneAquila di mareRays fishPesce chirurgoSurgeonfishLion fishpesce leonepesce scorpioneTartarugheturtlesmantasnorkelinggopro hero 7 blackgopro hero 4 black', 'In this video you can see the Whale shark Sharks Manta Stingray Surgeonfish Lion fish Turtles Rays fish and much moreFilmed with GoPro Hero 7 Black and GoPro Hero 4 BlackIn questo video potrete vedere lo squalo balena diversi squali tartarughe pesci chirurgo mante trigone aquile di mare e molto altroFilmato con GoPro Hero 7 Black e GoPro hero 4 Black', 'Ari Atol Maldive', 'Maldives']
Naive match [[], []]
Spacy match [['thudufushi'], ['sharks manta stingray'], []]

['53fd808d-9759-47fc-ac18-63a4e3715afc', 'https://www.youtube.com/watch?v=sziYTviQa74', 'Requin baleine  Whale shark', '', '

### Better version of matchLocation()
Function has the same stats as previous (coz it does the same stuff)

But now it works only with spacy and parses all items (not only YouTube videos)

In [7]:
def matchLocationV2(data, output=False):
    processed, identified = 0,0
    
    for item in data:
        locations = []
        
        # Filtering only items with title/descr/tags
        if item[2] or item[3] or item[4]:
        
            # Processing with spacy
            locations += nlpLocation(item[2].replace("'","")) # Title
            locations += nlpLocation(item[3].replace("'","")) # Tags
            locations += nlpLocation(item[4].replace("'","")) # Descr

            # Removing duplicates from found locations
            locations = list(set(locations))
            
            # Statistics
            processed += 1
            identified += 1 if len(locations) > 0 else 0
            
            if output:
                print(item)
                print(locations)
                print()
            
    print("Processed", processed, "entries")
    print("Identified", identified, "entries")
        
matchLocationV2(data, output=True)

['1910200691010', '', '', '', 'No other details known The pictures were taken with a Sea  Sea MX10 Camera so I dont know if theyre going to be clear enough for you', 'off the Coast of Coral Bay Western Australia when out on a Charter', '1a2']
[]

['191020116527', '', '', '', 'whale shark come to fishermans boat and feed flankton it was trained like aquarium', 'oslob cebu philippine', '6a2']
[]

['19112007171456', '', '', '', 'He came to us  he circled our snorketling day tour boat several times before we anchored and jumped in to meet him  After we played with him her and reboarded the boat he circled us again and headbutt the boat before he swam off encounter was at least 1 hour', 'Desecheo Island Puerto Rico', '2']
[]

['1911200723313', '', '', '', 'the guide state that these shark migrate thru these water every year from november till february', 'Golf of tadjoura djibouti africa', 'Djibouti']
[]

['191120078341', '', '', '', 'Small whale shark cruising over the reef', 'White Sands r

['193201061631', '', '', '', 'Was bitten on the right side fin', 'Djibouti East Africa  Arta Beach', 'Djibouti']
[]

['193201245054', '', '', '', 'White mark on top of head  Didnt look like a scar maybe some kind of marking or fungus', 'Donsol Sorsogon Philippines', '6a1']
[]

['193201402114', '', '', '', 'Source video ', 'Koh Ha Thailand', 'Thailand']
[]

['20420061719', '', '', '', 'ragged lower caudal fin small scar', 'Ningaloo Marine Park Coral Bay', '1a2']
[]

['1932014152327', '', '', '', 'first dorsal appears to be very falcate', 'East Contoy Island Mexico', '2c']
[]

['1932014222014', '', '', '', 'Source video ', 'Richelieu Rock Thailand', 'Thailand']
[]

['194200502135', '', '', '', 'Propellor scarring to the upper lobe of caudal fin  recent', 'Donsol Philippines', '6a1']
[]

['194200595713', '', '', '', 'Only one encounter recorded for this date on log sheet but appearance of two different sharks water quality change could be  same as 194200595113', 'Ningaloo Marine Park', '1

['19620100044', '', '', '', 'Nicks on caudal acoustic transmitter on right flank', 'East Contoy Mexico', '2c']
[]

['1982010244', '', '', '', 'Scrape on left flank scar on right side gill', 'Tofo Mozambique', '4a1']
[]

['1962010232711', '', '', '', 'Whale shark encountered while suba diving was first swimming with one group of divers then swam over and checked out the other group of divers while slowly swiming around them  Encountered at about 12 m went up to about 9 m and down again to 12 m', 'Ningaloo Reef', '1a']
[]

['1962013114119', '', '', '', 'Source video ', 'Placencia Belize', '2b']
[]

['1962013105947', '', '', '', 'acoustic tag 6', 'North Ningaloo', '1a1']
[]

['1972006233412', '', '', '', 'Sighted the whaleshark whilst a passenger on the Drawcard 3 Islands Marine Company', 'Ningaloo Reef  exmouth', '1a1']
[]

['196efd76-3df3-476b-b102-b25361108eb6', '', '', '', 'no interaction not shy but not friendly as a whaleshark use to be Only feeding opening his mouth and turning aro

['1d2e7f61-5b04-47b6-9052-b6a86c201571', '', '', '', 'BLA1514', 'Baha de los ngeles Baja California Mxico', '7d']
[]

['1da4afff-058a-48b3-b5c7-2d00e9b0c421', '', '', '', 'Noted to be a very small individual', 'Manta Point Nusa Penida Bali', 'Indonesia']
[]

['1e0df27a-13cb-4348-9697-c3632c5aab12', '', '', '', 'BLA14143', 'Baha de los ngeles Baja California Mxico', '7d']
[]

['1e2088eb-2ad4-4de8-be30-3959def45b8b', '', '', '', 'BLA1522', 'Baha de los ngeles Baja California Mxico', '7d']
[]

['1e32fedc-8f81-4d4e-8d2e-d70ba3f1ef21', '', '', '', 'Defecated during encounter Pic P2040079', 'St Helena', '2h1']
[]

['1e55d0c1-a425-43c0-b022-302c3192b6ec', '', '', '', 'Had an acoustic tag on it', 'Mafia Island Tanzania', 'Tanzania']
[]

['1e9b1ccd-063d-494e-ba5f-2665dacbfaa9', '', '', '', 'Source video ', 'Oslob Cebu Philippines', '6a2']
[]

['1eb71bc6-3977-4726-a15b-b10fde4aa756', '', '', '', 'Spotted on Ocean Eco Adventures Tour', 'Ningaloo Reef', '1a1']
[]

['1ee97fb8-f7cc-4213-aca9-ca8231b

['2042005133135', '', '', '', 'Saw about 67 seperate whale sharks throughout trip out to dive site one whilst on scuba and this one on return from diveSee also encounter 2872007231041', 'Mozambique  Inhambane  Off Barra Lodge  Dive Site Manta Reef', '4a1']
[]

['2042008182750', '', '', '', 'It stayed playing in our overboard discharge salt water for more than one hour was seen for about 3 to 4 hours around the platform and this is the third daily sighting in 3 consecutive days Moved off when our supply boat arrived', 'Approx 90km E of Bhubaneswar NE India', 'null']
[]

['204200863543', '', '', '', 'Body', 'Donsol Sorsogon', '6a1']
[]

['2042010164447', '', '', '', 'Tip of caudal fin missing flatten  Also carrying 2 tags', 'Utila Honduras', '2a']
[]

['204200925121', '', '', '', 'nick on dorsal fin', 'Donsol Sorsogon', '6a1']
[]

['20420092573', '', '', '', 'Fresh propeller cut on dorsal fin', 'Donsol Sorsogon', '6a1']
[]

['204200931825', '', '', '', 'propeller cut on dorsal fin', 'Don

...

['17220102194', '', '', '', 'Swimming in shallow 56 m muddy water adjacent to mangroves Visbility 2m Accompanied by several juvenile trevally Active feeding', 'Pantai Bentar East Java Indonesia', 'Indonesia']
[]

['1722010221651', '', '', '', 'Swimming in shallow 65 m muddy water adjacent to mangroves Visbility 2m Actively feeding', 'Pantai Bentar East Java Indonesia', 'Indonesia']
[]

['172201022325', '', '', '', 'Swimming in shallow 65 m muddy water adjacent to mangroves Visbility 2m Actively feeding Accompanied by several juvenile trevally remora', 'Pantai Bentar East Java Indonesia', 'Indonesia']
[]

['172201022655', '', '', '', 'Swimming in shallow 65 m muddy water adjacent to mangroves Visbility 2m Actively feeding Accompanied by several juvenile trevally remora', 'Pantai Bentar East Java Indonesia', 'Indonesia']
[]

['172201104917', '', '', '', 'nicks on trailing edge of lef pectoral', 'East Contoy Mexico', '2c']
[]

['1722010231042', '', '', '', 'Swimming in shallow 65 m muddy 

['176201019314', '', '', '', 'Left pectoral with bitelike rounded missing on trailing edge tail tip bended to the left Spot5 real time satellite transmitter trolling from left flank acoutic tranmitter on right side devices deployed by Rachel Graham and Dan Castellanos', 'East Contoy Mexico', '2c']
[]

['1762010234138', '', '', '', 'left pectoral with dark skin patches', 'East Contoy Mexico', '2c']
[]

['176201174122', '', '', '', 'Our guide thought he was about 8 years old between 4 and 5 meters long', 'Ningaloo Reef Western Australia', '1a']
[]

['176201265258', '', '', '', 'this whale shark swam very close to the surface and we could see his fin sticking out of the water before jumping in with him This female also swam nice and slow at the time so it was easy to follow her', 'Exmouth Ningaloo Reef Western Australia', '1a1']
[]

['17620127353', '', '', '', 'appeared to be interested in snorkelers and boat as it wasnt just swimming in a straight line It was doing turns and swimming tow

['179200825752', '', '', '', 'tofo beach inhambane province mozambique  Nick out of second dorsal fin', 'Tofo', '4a1']
['mozambique', 'inhambane', 'tofo']

['179200831242', '', '', '', 'tofo beach inhambane province mozambique', 'Tofo', '4a1']
['mozambique', 'inhambane', 'tofo']

['17920083331', '', '', '', 'tofo beach inhambane province mozambique  scars also on top tail and right head resight from 010208', 'Tofo', '4a1']
['mozambique', 'inhambane', 'tofo']

['17920086149', '', '', '', 'other aacrring and GPS is 23 86 384 which i cannot putin the space  sighted tofo beach inhamabne province mozambique', 'Tofo', '4a1']
['mozambique', 'tofo']

['17920087247', '', '', '', 'large chunk taken out of tail and GPS is 23 89857which i cannot putin the space  sighted tofo beach inhamabne province mozambique', 'Tofo', '4a1']
['mozambique', 'tofo']

['179200883312', '', '', '', 'Very inquisitive shark swining at divers and into them even changing direction to swim at the nearest person', 'Chumpho

['18320116018', '', '', '', 'Fast swims near the surface', 'Arta Beach Goulf of Tadjoura Djibouti', 'Djibouti']
[]

['183201162723', '', '', '', 'Very young specimen swim very fast', 'Billal Bay Goulf of Tadjoura Djibouti', 'Djibouti']
[]

['18320116731', '', '', '', 'Swims and eat near the surface', 'Billal Bay Goulf of Tadjoura Djibouti', 'Djibouti']
[]

['18320149405', '', '', '', 'Source video ', 'Tachai Pinnacle Thailand', 'Thailand']
[]

['1842009192350', '', '', '', 'Little nick on the caudal', 'Pumpkin Hill Utila Honduras', '2a']
[]

['1832014185728', '', '', '', 'Spotted on Surface Interval with UDC Dive Center Diveboat Old Tom', 'Northside of Utila', '2a']
[]

['18320149308', '', '', '', 'Source video ', 'Koh Lanta Thailand', 'Thailand']
[]

['183201493937', '', '', '', 'Source video ', 'Tachai Pinnacle Thailand', 'Thailand']
[]

['183201493950', '', '', '', 'Source video ', 'Tachai Pinnacle Thailand', 'Thailand']
[]

['1842004101557', '', '', '', 'This Shark has been seen th

['1882009145148', '', '', '', 'truncated tip of left pectoral', 'Ningaloo Marine Park Southern', '1a2']
[]

['18820103204', '', '', '', 'dorsal fin missing', 'Tofo Mozambique', '4a1']
[]

['18820105758', '', '', '', 'small v nicked out of first dorsal', 'Tofo Mozambique', '4a1']
[]

['18820107443', '', '', '', 'black and white scaring', 'Tofo Mozambique', '4a1']
[]

['1882011105257', '', '', '', 'The encounter occurred 35 miles from Holbox heading towards Isla Mujeres', 'Between Holbox and Isla Mujeres', '2c']
['holbox', 'mujeres']

['188201110544', '', '', '', 'The encountered occurred 35 miles from Holbox heading towards Isla Mujeres', 'Between Holbox and Isla Mujeres', '2c']
['holbox', 'mujeres']

['188201110550', '', '', '', 'Encounter occurred 35 miles from Holbox heading towards Isla Mujeres', 'Between Holbox and Isla Mujeres', '2c']
['holbox', 'mujeres']

['188201194157', '', '', '', 'Shark was encountered 35 miles from Holbox heading towards Isla Mujeres', 'Between Holbox and I

### Attempt to extract all identified locations from WEKA file to increase known locations dictionary

In [24]:
(data, attr) = loadWEKA('youTubeLocationIDWeka.csv', limit=0)
data[:2]

[['1892013124614', '', '', '', '', 'East Contoy Island Mexico', '2c'],
 ['1892013125327', '', '', '', '', 'East Contoy Island Mexico', '2c']]

In [27]:
location_names = []

for item in data:
    if not item[5].lower() in location_names:
        location_names.append(item[5].lower())

In [30]:
print("Found locations:", len(location_names))
location_names[:5]

Found locations: 4250


['east contoy island mexico',
 'red sea  marsa alam   dive elphinstone reef  north plateau',
 'red sea',
 'talisayan misamis oriental philippines',
 'black rock atoll tubbataha reef palawan']

In [14]:
from customPipeline import Pipe

ModuleNotFoundError: No module named 'customPipeline'